In [7]:
# Packages used

using DataFrames,
    CSV, 
    Statistics, 
    FinancialToolbox,
    Dates,
    Cascadia,
    Gumbo,
    HTTP

In [8]:
# Importing CSVs

amzn_data = CSV.read("amzn_itm_calls.csv", DataFrame)
amzn_itm_calls = CSV.read("amzn_data.csv", DataFrame)

,Column1,contractSymbol,lastTradeDate,strike,lastPrice,impliedVolatility
,Int64,String31,String31,Float64,Float64,Float64
1,0,AMZN240119C01580000,2022-01-21 15:26:57,1580.0,1358.05,0.407957
2,1,AMZN240119C01600000,2022-01-21 20:59:17,1600.0,1379.0,0.401018
3,2,AMZN240119C01620000,2022-01-21 19:39:42,1620.0,1393.7,0.392063
4,3,AMZN240119C01640000,2022-01-21 20:59:16,1640.0,1337.0,0.402312
5,4,AMZN240119C01660000,2022-01-21 20:45:11,1660.0,1331.03,0.388119
6,5,AMZN240119C01680000,2022-01-21 19:36:29,1680.0,1337.62,0.395745
7,6,AMZN240119C01700000,2022-01-21 19:35:54,1700.0,1246.45,0.387022
8,7,AMZN240119C01720000,2022-01-21 19:57:50,1720.0,1293.05,0.394589
9,8,AMZN240119C01740000,2022-01-21 16:45:32,1740.0,1325.0,0.384127


In [11]:
# Volatility Code

three_yrs = 252*3 
σ = std(last(amzn_data[!,"Returns"], three_yrs))*sqrt(252)

#  Dividend Code
d = 0

# Current price code

amzn_ex_price = last(amzn_data[!,"Adj Close"])
σ

0.29431845319986794

In [13]:
# risk_free_rate scraping

url = "https://www.bloomberg.com/markets/rates-bonds/government-bonds/us"
r = HTTP.get(url)
h = parsehtml(String(r.body))
body = h.root[2]
risk_free_raw_1 = eachmatch(Selector("#content > div > div >
        div.section-front__main-content > div.data-tables.first > div >
        table > tbody > tr:nth-child(6) > td:nth-child(4) > span"), body)

risk_free_raw_2 = (text(risk_free_raw_1[1]))

"1.72%"

In [14]:
# Risk free float conversion

risk_free_rate = replace(risk_free_raw_2, "%" => "" )
risk_free_float = parse(Float64, risk_free_rate)/100

0.0172

In [15]:
# Time delta code

today = Dates.today()
due_date = Dates.Date("2024-01-19")
T = (Dates.value(due_date - today))/252

2.876984126984127

In [16]:
function bstest(row)
    strikep = amzn_itm_calls[!,"strike"][row]
    blsp = blsprice(amzn_ex_price, strikep, risk_free_float, T, σ, d )
    actualp = amzn_itm_calls[!,"lastPrice"][row]
    println("Black-Scholes Price:", blsp)
    print("Actual Price:", actualp)
end

bstest (generic function with 1 method)

In [19]:
rownumber = size(amzn_itm_calls, 1)
randrow = rand(1:rownumber)
bstest(randrow)

Black-Scholes Price:1062.5263321233756
Actual Price:1109.44